In [1]:
# Activate Conda Env "Ultra" As the Kernel

import torch
import pandas as pd
from explainers.ego_util import batched_k_hop_subgraph
import networkx as nx
from pyvis.network import Network

In [2]:
dataset = torch.load('/storage/ryoji/Graph-Transformer/NBFNet-PyG/wn18rr_dataset.pt')
id2entity, id2relation = torch.load('/storage/ryoji/Graph-Transformer/NBFNet-PyG/wn18rr_id2name.pt')

In [3]:
# print(id2entity[19], id2relation[5], id2entity[40099])
# problem=12362

piciformes.n.01 _member_meronym piciform_bird.n.01


In [4]:
split = 'valid' # the split
ratio = 0.001 # the top r ratio of edges

if split == 'valid':
    data_index = 1
if split == 'test':
    data_index = 2

# output_file = f'/storage/ryoji/Graph-Transformer/NBFNet-PyG/explanation/NBFNet/WN18RR/explanation_output/{split}_output_hard_edge_mask_top_ratio_{ratio}.pt'
output_file = '/storage/ryoji/Graph-Transformer/NBFNet-PyG/RWExplainer/NBFNet/WN18RR/2024-12-03-17-33-34/valid_output_hard_edge_mask_top_k_100.pt'
# explanation_file = f'/storage/ryoji/Graph-Transformer/NBFNet-PyG/explanation/NBFNet/WN18RR/explanation_output/{split}_explanations_hard_edge_mask_top_ratio_{ratio}.pt'
explanation_file = '/storage/ryoji/Graph-Transformer/NBFNet-PyG/RWExplainer/NBFNet/WN18RR/2024-12-03-17-33-34/valid_explanations_hard_edge_mask_top_k_100.pt'
explanations = torch.load(explanation_file)
outputs = torch.load(output_file)
output_df = pd.DataFrame(outputs)

data = dataset[data_index]

In [5]:
def vizualize_explanation(index):
    row = output_df.iloc[index]
    if row['Mode'] == 1:
        rel = id2relation[row['Rel']]
    else:
        rel = id2relation[row['Rel']]+'_inv'

    head_id = row['Heads']
    tail_id = row['Tails']
    head = id2entity[head_id]
    tail = id2entity[tail_id]

    print(f"*** Query: {head, rel}. Answer: {tail}, Rank given the explanation: {row['Ranking']} ***")
    nodes, edges = batched_k_hop_subgraph(torch.tensor([head_id]).unsqueeze(0), 6, data.edge_index, data.num_nodes)

    print(f'Is the tail included in the 6-hop neighbor of head in the original graph? {nodes[0, tail_id].item()}')
    full_edge_index = data.edge_index[:, edges.squeeze()]

    expl = explanations[index]
    # Check that the explanation are restricted only to within the ego_network (not necessary)
    # assert torch.all(torch.logical_and(expl, ~edges.squeeze()) == False)
    expl_edge_index = data.edge_index[:, expl.to(torch.bool)]

    print(f'Is the head included in the Explanation? {torch.any((expl_edge_index == head_id)).item()}')
    print(f'Is the tail included in the Explanation? {torch.any((expl_edge_index == tail_id)).item()}')

    # Create a directed graph
    G = nx.DiGraph()

    # Add edges from expl_edge_index
    for i in range(expl_edge_index.shape[1]):
        source = expl_edge_index[0, i].item()
        target = expl_edge_index[1, i].item()
        # G.add_edge(source, target,arrows='to')
        G.add_edge(source, target)

    attrs = {head_id: "#FF0000", tail_id: "#00FF00"}
    nx.set_node_attributes(G, attrs, name='color')

    try:
        paths = list(nx.all_simple_paths(G, source = head_id, target=tail_id, cutoff = 6))
    
        attrs = {}
        if len(paths) > 0:
            connected = True
        else:
            connected = False
    
        print(f'Is the tail connected to the head? {connected}')
        for path in paths:
            src = head_id
            for node in path[1:]:
                edge = (src, node)
                attrs[edge] = {'color':"#b27ebd"}
                src = node

        nx.set_edge_attributes(G, attrs)
    except:
        print(f'Is the tail connected to the head? {False}')
            
    
    return G
    

In [6]:
display(output_df)

,Ranking,Heads,Tails,Rel,Mode,Num_Edges,Num_Nodes,Inclusion
0,3,1789,9534,5,1,100,32,1
1,1,35650,23897,1,1,100,31,1
2,41105,25316,19796,3,1,100,29,0
3,5,37750,4842,9,1,94,35,1
4,41105,25541,8811,3,1,100,36,0
...,...,...,...,...,...,...,...,...
6063,1,6497,16457,1,0,100,27,1
6064,11,5538,5814,2,0,100,30,1
6065,1,13787,2914,10,0,100,30,1
6066,41104,34493,36131,1,0,94,25,0


In [19]:
# Control the index (row number of output_df of the instance you want to inspect here)
index = 3

In [20]:
G = vizualize_explanation(index)

*** Query: ('mathematical_process.n.01', '_synset_domain_topic_of'). Answer: mathematics.n.01, Rank given the explanation: 5 ***
Is the tail included in the 6-hop neighbor of head in the original graph? True
Is the head included in the Explanation? True
Is the tail included in the Explanation? True
Is the tail connected to the head? True


In [21]:
# *** Explanation Graph Visualization ***
# RED NODE: The Query Head (if inside the explanation)
# GREEN NODE: The Answer Tail (if inside the explanation)
# Purple Edges: Edges that are in any path that connected Red to Green
net = Network(notebook = True, cdn_resources = "remote",
                    bgcolor = "#222222",
                    font_color = "white",
                    height = "750px",
                    width = "100%",
                    select_menu = True,
                    filter_menu = True,
                    directed=True
    )
net.from_nx(G)
net.inherit_edge_colors(False)
net.set_edge_smooth('dynamic')
net.show('graph.html')

graph.html
